
# RAID-5 #

- [RAID-5](#raid-5)
  - [Introduction](#introduction)
  - [Design and Implementation ](#design-and-implementation-)
    - [Client/Server Architecture](#clientserver-architecture)
    - [Load Distribution](#load-distribution)
    - [Fault Tolerance](#fault-tolerance)
      - [Parity](#parity)
      - [Checksum](#checksum)
    - [Recovery](#recovery)
      - [Block Recovery](#block-recovery)
      - [Server Recovery](#server-recovery)
      - [Failover](#failover)
  - [Evaluation ](#evaluation-)
  - [Reproducibility ](#reproducibility-)
    - [Failover and Server Failure](#failover-and-server-failure)
    - [Corrupt block](#corrupt-block)
  - [Conclusions ](#conclusions-)

## Introduction<a name="introduction"></a> ##

Describe in your own words what your project accomplishes, and motivate the decisions made

## Design and Implementation <a name="design"></a> ##

### Client/Server Architecture ###

### Load Distribution ###

There are a couple reasons to distribute the data across multiple servers -

1. Reduced server load by distributing requests
2. Increased fault tolerance

To achive data distribution the project employs RAID-5 methodology. RAID 5 distributes the parity along with the data across the servers available.

The client will be aware of the data blocks. The number of servers and the distribution of requests will be handled by the RAID 5 by mapping the client block number to a server number and physical block number.

The distribution pattern is shown in the Figure 1.

![Figure 1. Data and Parity Distribution](image-3.png)

Virtual to physical mapping mechanism is handled by two methods `MapVirtualBlockToPhysicalAddress` and `MapPhysicalAddressToVirtualBlock`. For example, given the virtual block number `10` the program will be able to map it with server `2` and block index `3` and vice-verca.

Both of the functions are designed to skip the parity blocks and make sure client only have access to data blocks.

![Figure 2. Data Strip - Participating Blocks in Recovery](image-5.png)

Data strip is defined as the set of blocks for a given block index across all of the servers, as shown in Figure 2. Each data strip contains a parity associated and stored on one of the servers.

The parity location is determined by the method `GetParityServer` given the block index. For example, for block index `3`, the program is able to determine the parity server i.e. `0`.

Mapping method also includes a helper `GetBlockNumberOnStrip` which provides all of the blocks available across a data strip. This is useful while determining the parity, block and server recovery procedures.

### Fault Tolerance ###

Project is designed to tolerate two types of failures. Both of the failures employ data recovery utilizing the other data blocks and the parity block on the given data strip.

1. Server Failure:
  
   The goal is to be able to continue to operate even after a complete server crashes. And when it recovers from the crash, a client should be able to recover the data using other blocks and parity on that data strip.

2. Corrupt Block:

    Here, a particular block data could get corrupt and we implement checksum to validate the data to ensure that the data is correct before sending it back to the client.

    If the data is corrupt we recover the data using the recovery method which includes the parity and other data blocks on the data strip.
  
#### Parity ####

Parity calculation is handled through `GetParity` method. There are two scenarios which are handled -

a. when the parity is being created for the first ever time for the data strip. In this case we fetch all of the data block on the strip and then calculate the parity.

```python
# GetParity()
# ...
# ...
  # create parity with data on the strip - data blocks
  strip = self.GetBlockNumberAcrossServers(block_idx)
  blocks = strip[0]
  data = self.FetchDataBlocks(blocks)
  # calculate and return parity 
  new_parity = self.CalculateParity(data)
# ...
# ...
```

b. When some parity already exist on the strip and certain block data is updated on the strip. Here, we could leverage the previously calculated parity and the new data to determine new parity -

```python
# GetParity()
# ...
# ...
  data = [old_data, new_data, old_parity]
  # calculate and return parity 
  new_parity = self.CalculateParity(data)
# ...
# ...
```

Parity is calculated using a bitwise xor operation handled in `CalculateParity` method.

#### Checksum ####

While writing to a block on the server, we keep a checksum, calculated using `hashlib.md5` method. and store in a dictionary.

```python
# PUT()
# ...
# ...
    block_checksum = hashlib.md5(data.data).hexdigest()
    # emulating corruption
    print('>>> Checksum Returned From PUT Server ', cblk, block_number)
    if block_number == cblk and RawBlocks.make_corrupt: 
       RawBlocks.make_corrupt = False
       block_checksum = block_checksum[:5] + '12345'
    RawBlocks.checksum[block_number] = block_checksum
# ...
# ...
```

This is then verified while reading the data and recalculating the checksum of the data and comparing it with the saved checksum. If any discrepancy is found, we report the error as `CHECKSUM_ERROR` on the client.

```python
# GET()
# ...
# ...
    has_error = False
    result = RawBlocks.block[block_number]
    current_checksum = hashlib.md5(result).hexdigest()
    # fetch checksum from cache
    block_checksum = RawBlocks.checksum.get(block_number)
    print('>>> Checksum Returned From GET Server ', block_checksum)
    if block_checksum != None and current_checksum != block_checksum:
      print('>>> Corrupt Block Detected: Checksum Mismatch')
      has_error = True
# ...
# ...
```

It takes a long time for a block to corrupt, to simulate this, `cblk` parameter is provided with the block index while spinning the server which is used to corrupt the checksum of the block.

### Recovery ###

#### Block Recovery ####

As per the previous discussion, the block recovery utilizes the other data blocks and the parity to recover from the corruption.

The process involves the following steps.

   1. determine the block_index of the corrupt block
   2. Fetch the data strip and the parity server
   3. Filter the corrupt block from strip
   4. Fetch teh data blocks and parity block
   5. Use parity function to recover the data

#### Server Recovery ####

Each server contains BLOCK_SIZE number of blocks and recovering data for all of the blocks involves recovering each of the blocks, including the parity blocks.

Along with block recovery function, additional parity block recovery procedure is written which is slighly different fromt he data block recovery.

![Figure 3. server #2 in failure state](image-4.png)

The process for parity recovery involves the following steps -

  1. determine the block index of the corrupt block
  2. Fetch the data strip
  3. use parity function to recover the data

The rest of the server recovery involves recovering each of the data and parity block one at a time.

```python
# RecoverServer()
# ..
# ..
  # virtual to physical will skip the parity servers while mapping virtual block numbers
  parity_block_physical_ids = [fsconfig.MAX_SERVERS - int(server_id) - 1 + idx * 4 for idx in range(0, 64)]

  data_block_physical_ids = [idx for idx in range(0, 192) if idx not in parity_block_physical_ids]
  data_block_virtual_ids = [self.MapPhysicalAddressToVirtualBlock(int(server_id), idx) for idx in data_block_physical_ids]

  # recover data blocks - RecoverDataBlock()
  for block in zip(data_block_physical_ids, data_block_virtual_ids):
     recovered_block_data = self.RecoverDataBlock((int(server_id), block[0]), block[1])
     self.Put(block[1], recovered_block_data)

  # recover parity blocks - RecoverParityBlock()
  for block_idx in parity_block_physical_ids:
     recovered_parity_data = self.RecoverParityBlock(block_idx)
     self.SinglePut(int(server_id), block_idx, recovered_parity_data)
# ..
# ..
```

We use the SinglePut method for parity recover instead of the Put for data recovery, because the Put method utilizes the virtual to physical mapping - which skips the parity blocks.

#### Failover ####

While the server is in failure state, the client will try to read the data and it will log `Server_Disconnected` error but without failing the command.

The read operations will continue to work by recovering the data utilizing other blocks on the strip.

## Evaluation <a name="evaluation"></a> ##

Describe how you tested your program and, for EEL-5737 students, how you evaluated the load distribution.

## Reproducibility <a name="reproducibility"></a> ##

Running the project:

To run a server: `python3 ./blockserver.py -nb 256 -bs 128 -port <start_port>`

The server ports numbers should differ by atmost by 1.

After all of the servers are up and running. We connect a client -

To run the client: `python3 ./fsmain.py -port 8000 -cid 0 -startport start_port -ns 4 -nb total_data_blocks`

Total number of blocks = `-nb = (number_of_servers - 1) * (number_of_block_size_each_server)`

### Failover and Server Failure ###

1. Create a file through client
2. Stop server #2
3. Try to locate the file using `ls` 
4. `Server Disconnected` message should be printed
5. However, client will not fail and file will still show up
6. Restart the server #2
7. run `repair 2` on client and wait until the command is executing
8. No error message will be printed
9. client will resume normal operations

### Corrupt block ###

1. For any one of the servers we add additional `-cblk block_number` argument.
2. Connect the client
3. command

## Conclusions <a name="conclusions"></a> ##
